# Part 2: Joining a Secondary Dataset

In this notebook, I join my original Netflix IMDB dataset with a secondary dataset I created with the help of AI.

The join key is imdb_id.

I made the secondary dataset by copying the imdb_id column into a new csv file and then generating random values for the other columns.


In [27]:
import pandas as pd


## Load the original NetflixIMDB dataset

This file contains Netflix titles and their IMDB scores.


In [28]:
netflix = pd.read_csv("../data/NetflixIMDB.csv")
netflix.head()


,index,id,title,type,description,release_year,age_certification,runtime,imdb_id,imdb_score,imdb_votes
0,0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,tt0075314,8.3,795222.0
1,1,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,tt0071853,8.2,530877.0
2,2,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,tt0079470,8.0,392419.0
3,3,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,tt0070047,8.1,391942.0
4,4,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,tt0063929,8.8,72895.0


# Checking Dataset
I just ran these to make sure my original dataset was loaded in properly

In [29]:
print("Netflix shape:", netflix.shape)
print("Netflix columns:")
print(list(netflix.columns))


Netflix shape: (5283, 11)
Netflix columns:
['index', 'id', 'title', 'type', 'description', 'release_year', 'age_certification', 'runtime', 'imdb_id', 'imdb_score', 'imdb_votes']


## Load the secondary dataset

This dataset was manually generated in Excel using the same imdb_id that was in the original.

I added columns like genre, country, language, popularity bucket, and award status.

I used Claude to help me with the data generating instructions


In [30]:
secondary = pd.read_csv("../data/netflix_secondary.csv")
secondary.head()


,imdb_id,primary_genre,production_country,language,popularity_bucket,is_award_winner
0,tt0075314,Romance,ES,Hindi,High,No
1,tt0071853,Action,UK,Spanish,Low,No
2,tt0079470,Family,IN,Korean,Low,No
3,tt0070047,Horror,DE,Japanese,High,No
4,tt0063929,Drama,DE,German,Low,Yes


# Checking Secondary Dataset

In [31]:
print("Secondary shape:", secondary.shape)
print("Secondary columns:")
print(list(secondary.columns))


Secondary shape: (5283, 6)
Secondary columns:
['imdb_id', 'primary_genre', 'production_country', 'language', 'popularity_bucket', 'is_award_winner']


## Removing duplicate imdb_id values 

If the secondary dataset accidentally has duplicate IDs, it could create repeated rows in the merge.


In [32]:
secondary = secondary.drop_duplicates(subset="imdb_id")
print("Secondary shape after dropping duplicates:", secondary.shape)


Secondary shape after dropping duplicates: (5283, 6)


## Merge the datasets (LEFT join)

I used a LEFT merge because I wanted to keep every row from the original Netflix dataset.
If a title does not have a match in the secondary dataset, the new columns will become NA.


In [33]:
merged = pd.merge(netflix, secondary, how="left", on="imdb_id")
merged.head()


,index,id,title,type,description,release_year,age_certification,runtime,imdb_id,imdb_score,imdb_votes,primary_genre,production_country,language,popularity_bucket,is_award_winner
0,0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,tt0075314,8.3,795222.0,Romance,ES,Hindi,High,No
1,1,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,tt0071853,8.2,530877.0,Action,UK,Spanish,Low,No
2,2,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,tt0079470,8.0,392419.0,Family,IN,Korean,Low,No
3,3,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,tt0070047,8.1,391942.0,Horror,DE,Japanese,High,No
4,4,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,tt0063929,8.8,72895.0,Drama,DE,German,Low,Yes


# Checking the merged data

In [34]:
print("Merged shape:", merged.shape)


Merged shape: (5283, 16)


## Saving the merged dataset

I save the merged dataset into the clean_data folder.

This file is not uploaded to GitHub because clean_data is in .gitignore.


In [35]:
merged.to_csv("../clean_data/netflix_merged.csv", index=False)